In [1]:
%run oeai_py

StatementMeta(, , -1, Finished, Available)

In [2]:
oeai = OEAI(platform="Fabric")

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 4, Finished, Available)

In [3]:
# CHANGE VALUES FOR YOUR KEY VAULT
keyvault = "https://kv-oea-uniteddata.vault.azure.net/"  
keyvault_linked_service = ""  

# Synapse OEA environment paths
bronze_path = oeai.get_secret(spark, "arbor-Bronze", keyvault_linked_service, keyvault)
silver_path = oeai.get_secret(spark, "arbor-Silver", keyvault_linked_service, keyvault)
#gold_path = oeai.get_secret(spark, "gold-path", keyvault_linked_service, keyvault)
#school_ids_secret = oeai.get_secret(spark, "arbor-ids", keyvault_linked_service, keyvault)
#subdirectories = school_ids_secret.split(",")

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 5, Finished, Available)

In [4]:
# Define the mapping between JSON files and desired Delta table names
delta_table_name_mapping = {
    "schools.csv": "dim_Organisation",
    "students.csv": "dim_Student",
    "students_extended.csv": "dim_StudentExtended",
    "attendancesession.csv": "fact_AttendanceSession",
   
}

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 6, Finished, Available)

In [5]:
column_mappings = {
    "schools.csv": {
        # drops
        "ABBREVIATED_NAME": "drop",
        #"school_id": "drop",
        # Renames
        "APPLICATION_ID": {"new_name": "school_id"}, 
        "NAME": {"new_name": "Organisation_Name"},    
        "PHASE": {"new_name": "Organisation_Type"},
        # adds
        "add_columns": {
            "organisationkey": "",  
            "addresskey": "",  
            "UKPRN": "",
            "Organisation_Status": "Active",
            "last_updated": "",
            "Establishment_Number": "",
            "URN": "",
            "LA_Code": "",
            "unique_key": "",
        }
    },
    "students.csv": {
        # drops   
        "STUDENT_UNIQUE_ID": "drop",
        #"UPN": "drop",
        "IS_CURRENT": "drop",
        "SEX": "drop",
        "ETHNICITY": "drop",
        "RELIGION": "drop",
        "NATIVE_LANGUAGE_CODES": "drop",
        "NATIVE_LANGUAGE_NAMES": "drop",
        "EMAIL_ADDRESS": "drop",
        "NATINALITIES": "drop",
        "CURRENT_YEAR_GROUP": "drop",
        "CURRENT_SEN_STATUS": "drop",
        "CURRENT_REGISTRATION_GROUP": "drop",
        "CHILD_PROTECTION": "drop",
        "GIFTED": "drop",
        "GIFTED_TALENTED": "drop",
        "HAS_MEDICAL_CONDITION": "drop",
        "IN_CARE": "drop",
        "IN_YEAR_ADMISSION": "drop",
        "OUT_OF_AGE_GROUP_COHORT": "drop",
        "SEN": "drop",
        "TALENTED": "drop",
        "EAL": "drop",
        "COMPULSORY_SCHOOL_AGE": "drop",
        "EVER_6-FSM": "drop",
        "EARLY_YEARS_PUPIL_PREMIUM_RECIPIENT": "drop",
        "EVER_6SERVICE_CHILD": "drop",
        "FSM": "drop",
        "MOBILE_Y10_Y11": "drop",
        "MOBILE_Y5_Y6": "drop",
        "PUPIL_PREMIUM": "drop",
        "PUPIL_PREMIUM_RECIPIENT": "drop",
        "SERVICE_CHILD": "drop",
        "TRAVELLER": "drop",
        "DISADVANTAGED": "drop",
        # Renames
        "UPN": {"new_name": "UPN"}, 
        "DATE_OF_BIRTH": {"new_name": "Date_Of_Birth"}, 
        "LEGAL_FIRST_NAME": {"new_name": "Forename"}, 
        "GENDER": {"new_name": "Gender"}, 
        "STUDENT_ID": {"new_name": "student_id"}, 
        "LEGAL_FIRST_NAME": {"new_name": "Legal_Forename"}, 
        "LEGAL_LAST_NAME": {"new_name": "Legal_Surname"}, 
        "PREFERRED_LAST_NAME": {"new_name": "Surname"},
        "PREFERRED_FIRST_NAME": {"new_name": "Forename"},       
        "APPLICATION_ID": {"new_name": "school_id"},
        # adds
        "add_columns": {
            "organisationkey": "",
            "studentkey": "",
            "Middle_Names":"",
            "addresskey":"",
            "ULN":"",
            "last_updated":"",
            "unique_key": "",
        }
    },
    "students_extended.csv": {
        # drops
        
        "STUDENT_UNIQUE_ID": "drop",
        
        "UPN": "drop",
        "LEGA_FIRST_NAME": "drop",
        "LEGA_LAST_NAME": "drop",
        "PREFERRED_LAST_NAME": "drop",
        "PREFERRED_FIRST_NAME": "drop",
        "DATE_OF_BIRTH": "drop",
        "GENDER": "drop",
        "NATIVE_LANGUAGE_CODES": "drop",
        "EMAIL_ADDRESS": "drop",
        "CURRENT_YEAR_GROUP": "drop",
        "CURRENT_REGISTRATION_GROUP": "drop",
        "GIFTED": "drop",
        "HAS_MEDICAL_CONDITION": "drop",
        "IN_YEAR_ADMISSION": "drop",
        "OUT_OF_AGE_GROUP_COHORT": "drop",
        "SEN": "drop",
        "TALENTED": "drop",
        "COMPULSORY_SCHOOL_AGE": "drop",
        "EARLY_YEARS_PUPIL_PREMIUM_RECIPIENT": "drop",
        "EVER_6_SERVICE_CHILD": "drop",
        "MOBILE_Y10_Y11": "drop",
        "MOBILE_Y5_Y6": "drop",
        "TRAVELLER": "drop",
        "DISADVANTAGED": "drop",
        # Renames
        "EAL": {"new_name": "English_As_Additional_Language"}, 
        "IS_CURRENT": {"new_name": "Enrolment_Status"}, 
        "ETHNICITY": {"new_name": "Ethnicity"},
        "RELIGION": {"new_name":"Religion"},
        "NATIONALITIES": {"new_name": "Nationality"},
        "IN_CARE": {"new_name": "In_Care"}, 
        "NATIVE_LANGUAGE_NAMES": {"new_name": "First_Language"}, 
        "FSM": {"new_name": "Free_School_Meals"}, 
        "EVER_6_FSM": {"new_name": "Free_School_Meals_6"}, 
        "GIFTED_TALENTED": {"new_name": "Gifted_And_Talented_Status"}, 
        "PUPIL_PREMIUM_RECIPIENT": {"new_name": "Pupil_Premium_Indicator"}, 
        "PUPIL_PREMIUM": {"new_name": "Pupil_Premium_Eligible"}, 
        "CURRENT_SEN_STATUS": {"new_name": "SEN_Status"}, 
        "SERVICE_CHILD": {"new_name": "Service_Child"},            
        "CHILD_PROTECTION": {"new_name": "Child_Protection_Plan"},         
        "STUDENT_ID": {"new_name": "student_id"}, 
        "APPLICATION_ID": {"new_name": "school_id"}, 
        # adds
        "add_columns": {
            "organisationkey": "",
            "studentextendedkey": "",
            "studentkey": "",
            "Ethnicity_Code":"",
            "In_LEA_Care" :"", 
            "Leaver_Destination" :"",
            "Ever_In_Care": "",
            "Enrolment_Status": "",
            "Child_In_Need": "",    
            "Leaver_Destination": "",
            "In_LEA_Care": "",
            "Current_Year": "",
            "NC_Year": "",
            "unique_key": "",
        }
    },
        "attendancesession.csv": {
        # drops
        "ATTENDANCE_ROLL_CALL_RECORD_ID":"drop",
        "STUDENT_UNIQUE_ID":"drop",
        "APPLICATION_ID":"drop",
        "ATTENDANCE_ROLL_CALL_ID":"drop",
        "ATTENDANCE_ROLL_CALL_RECORD_UNIQUE_ID":"drop",
        "MARK_CODE":"drop",
        "MINUTES_LATE":"drop",
        "ACADEMIC_YEAR_NAME":"drop",
        "IS_PRESENT":"drop",
        "IS_AUTHORIZED_ABSENT":"drop",
        "IS_UNAUTHORIZED_ABSENT":"drop",
        "IS_POSSIBLE_ATTENDANCE":"drop",
        # Renames 
        "DATE": {"new_name": "Date"},  
        "ATTENDANCE_ROLL_CALL_UNIQUE_ID": {"new_name": "school_id"}, 
        "PERIOD": {"new_name": "Session"}, 
        "RAW_MARK": {"new_name": "Mark"},
        "STUDENT_ID": {"new_name": "student_id"}, 
        # adds
        "add_columns": {
            "organisationkey": "",
            "attendancesessionkey": "",
            "studentkey": "",
            "Comment": "",
            "staff_id": "",
            "last_updated": "",
            "unique_key": "",
        }
    }
}

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 7, Finished, Available)

In [6]:
# Dictionary to hold dataframes for each csv file
csv_dfs = {}
temp_dfs = {}
all_columns = {}
'''
    This code is to loop through each csv file to compile into
    a list of dfs to process

    It creates csv_dfs - a dictionary of the csv files
'''
# Consider only csv files that are in your mapping
csv_files = list(delta_table_name_mapping.keys())

#print(list(delta_table_name_mapping.keys()))
for csv_file in csv_files:
    csv_file_path = f"{bronze_path}/{csv_file}"
    try:
        temp_df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
        # Update the set of columns for the csv_file
        all_columns.setdefault(csv_file, set()).update(temp_df.columns)
        temp_dfs[csv_file] = temp_df
    except Exception as e:
        print(f"An unexpected error occurred while processing {csv_dir_path}: {e}")
        continue
# Assign the final json_dfs outside the loops
csv_dfs = temp_dfs

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 8, Finished, Available)

In [7]:
for csv_name, df in csv_dfs.items():
    if csv_name in column_mappings:
        df = oeai.apply_column_mappings(df, column_mappings[csv_name])
        csv_dfs[csv_name] = df  

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 9, Finished, Available)

In [8]:
# add all the unique_keys
# schools.csv
df_schools = []
df_schools = csv_dfs['schools.csv']
df_schools = df_schools.withColumn("unique_key", df_schools["school_id"])

# students.csv
df_stud = []
df_stud = csv_dfs['students.csv']
df_stud = df_stud.withColumn("unique_key", concat_ws("_",df_stud["school_id"],df_stud["student_id"] ))

# students_extended.csv
df_studext = []
df_studext = csv_dfs['students_extended.csv']
df_studext = df_studext.withColumn("unique_key", concat_ws("_",df_studext["school_id"],df_studext["student_id"] ))

# attendance.csv
df_att = []
df_att = csv_dfs['attendancesession.csv']
df_att = df_att.withColumn("unique_key", df_att["school_id"])

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 10, Finished, Available)

"\n# List of jobs next to get the dimensions in the correct schema.\n\n## need to sort schools out to populate URN from external_id\n\n\n# dim_Student requires UPN and Current_Year from StudentEducation\n# Access the DataFrames\ndf_joined = []\ndf_student = csv_dfs['students.csv']\ndf_student_attendance = csv_dfs['students_education.json']\n\ndf_joined = df_student.join(\n    df_student_attendance.select('student_id', 'UPN', 'Current_Year'),\n    on='unique_key',  # column name to join on, which must be present in both DataFrames\n    how='inner'  # you can also use 'left', 'right', or 'outer' as needed\n)\njson_dfs['students.json'] = df_joined\n\n\n#class and group union\n#df_classes = json_dfs['classes.json']\n#df_groups = json_dfs['groups.json']\n#df_classes.printSchema()\n#df_groups.printSchema()\n#df_union = df_classes.unionByName(df_groups)\n\n#json_dfs['groups.json'] = df_union\n"

In [9]:
# Process each DataFrame and upsert it to the silver_path
for csv_name, df in csv_dfs.items():
    
    if csv_name in delta_table_name_mapping and delta_table_name_mapping[csv_name] != "":
        # Get the Delta table name from the mapping
        delta_table_name = delta_table_name_mapping[csv_name]
        silver_table_path = f"{silver_path}/{delta_table_name}"
        uuid_column_name = oeai.get_uuid_column_name(delta_table_name)

        # Define the unique key column name
        unique_key_column = "unique_key"  

        if delta_table_name == "dim_Organisation":
            if DeltaTable.isDeltaTable(spark, silver_table_path):
                delta_table = DeltaTable.forPath(spark, silver_table_path)
                
                # Alias the Delta table as 'target' and rename 'organisationkey' to 'target_organisationkey'
                target_df = delta_table.toDF().select(unique_key_column, col("organisationkey").alias("target_organisationkey"))
                
                # Alias the source DataFrame as 'source'
                source_df = df.alias("source")
                
                # Perform a left join to find non-matched records
                df_with_keys = source_df.join(
                    target_df,
                    source_df[unique_key_column] == target_df[unique_key_column],
                    how="left"
                ).select(
                    # Select all columns from 'source' EXCEPT 'organisationkey' if it exists
                    *[source_df[col].alias(col) for col in source_df.columns if col != "organisationkey"],
                    # Coalesce to get 'organisationkey' from 'target' if it exists, or generate a new one
                    coalesce(col("target_organisationkey"), expr("uuid()")).alias("organisationkey")
                )
                
                # Now perform the merge operation
                delta_table.alias("target").merge(
                    df_with_keys.alias("source"),
                    f"target.{unique_key_column} = source.{unique_key_column}"
                ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
                
            else:
                # If the table does not exist, create it by writing the current DataFrame
                # First, add a column for the organisationkey for all records since this is a new table
                df = df.withColumn("organisationkey", expr("uuid()"))
                df.write.format("delta").mode("overwrite").save(silver_table_path)
        else:
            # Process student table before all others:
            if delta_table_name == "dim_Student":
                if DeltaTable.isDeltaTable(spark, silver_table_path):
                    delta_table = DeltaTable.forPath(spark, silver_table_path)
                    try:
                        update_columns = {col: f"source.{col}" for col in df.columns if col not in ['organisationkey', uuid_column_name]}
                        delta_table.alias("target").merge(
                            df.alias("source"),
                            f"target.{unique_key_column} = source.{unique_key_column}"
                        ).whenMatchedUpdate(set=update_columns  # Use the dictionary of columns to update
                        ).whenNotMatchedInsertAll().execute()
                    except Exception as e:
                        print(delta_table_name)
                        df.printSchema()
                        print(e)
                else:
                    # If the table does not exist, create it by writing the current DataFrame
                    df = df.withColumn(uuid_column_name, expr("uuid()"))
                    # Load the dim_Organisation table to get the existing mappings
                    dim_org_df = spark.read.format("delta").load(f"{silver_path}/dim_Organisation").select("school_id", "organisationkey")
                    # Perform a left join to find existing organisation keys
                    df_joined = df.alias("source").join(
                        dim_org_df.alias("dim"),
                        col("source.school_id") == col("dim.school_id"),
                        "left"
                    )
                    # Select all columns from df and only the 'organisationkey' from the dim_Organisation
                    df_with_keys = df_joined.select("source.*", col("dim.organisationkey").alias("dim_organisationkey"))
                    # Fill in the missing keys with UUIDs
                    df_complete = df_with_keys.withColumn(
                        "organisationkey",
                        when(col("dim_organisationkey").isNull(), expr("uuid()")).otherwise(col("dim_organisationkey"))
                    )
                    # Drop the 'dim_organisationkey' as it is no longer needed
                    df_final = df_complete.drop("dim_organisationkey")
                    df_final.write.format("delta").mode("overwrite").save(silver_table_path)
            else: # if not the organisation or student table         
                
                if ('studentkey' in df.columns) and (delta_table_name != "dim_Student"):
                    # ------------------------------
                    # First, get the organisationkey
                    # ------------------------------
                    # Read the dim_Organisation table
                    dim_org_df = spark.read.format("delta").load(f"{silver_path}/dim_Organisation").select("school_id", "organisationkey")
                    
                    df = df.drop("organisationkey")

                    # Perform a left join to find existing organisation keys
                    df_joined = df.alias("source").join(
                        dim_org_df.alias("dim"),
                        col("source.school_id") == col("dim.school_id"),
                        "left"
                    )

                    # df_joined = df.drop("external_id")

                    # Select all columns from df (source) and only the 'organisationkey' from dim_Organisation (dim)
                    # Alias the dim_Organisation's organisationkey to avoid ambiguity
                    
                    df_with_orgkey = df_joined.select(
                        *[col(f"source.{col_name}") for col_name in df.columns],
                        col("dim.organisationkey")
                    )

                    # --------------------------
                    # second, get the studentkey
                    # --------------------------
                    
                    dim_student_df = spark.read.format("delta").load(f"{silver_path}/dim_Student").select("student_id", "organisationkey", "studentkey")
                    # Rename the 'studentkey' column from dim_student_df to avoid ambiguity
                    dim_student_df = dim_student_df.withColumnRenamed("studentkey", "dim_studentkey")
                    
                    # Perform a left join
                    df_studjoined = df_with_orgkey.alias("source").join(
                        dim_student_df.alias("dim"),
                        (trim(lower(col("source.student_id"))) == trim(lower(col("dim.student_id")))) &
                        (trim(lower(col("source.organisationkey"))) == trim(lower(col("dim.organisationkey")))),
                        "left"
                    )
                    
                    # Use when() to decide which studentkey to keep
                    df_both_keys = df_studjoined.withColumn("studentkey", 
                                            when(col("dim.dim_studentkey").isNull(), col("source.studentkey"))
                                            .otherwise(col("dim.dim_studentkey"))
                                            ) \
                                .drop("dim.dim_studentkey") \
                                .select("source.*", "studentkey")

                    df = df_both_keys

                    # debug, show 20 records
                    #df.filter((col("studentkey").isNotNull()) & (col("studentkey") != "")).select("studentkey").show(n=20, truncate=False)

                # -------------------------------------------------------------------
                # Now that any table with student_id in it has studentkey continue...
                # -------------------------------------------------------------------

                # Set the update columns to update everything other than organisationkey and the unique_key
                update_columns = {col: f"source.{col}" for col in df.columns if col not in ['organisationkey', uuid_column_name]}
                # print(update_columns)

                if DeltaTable.isDeltaTable(spark, silver_table_path):
                    delta_table = DeltaTable.forPath(spark, silver_table_path)
                    try:
                        delta_table.alias("target").merge(
                            df.alias("source"),
                            f"target.{unique_key_column} = source.{unique_key_column}"
                        ).whenMatchedUpdate(set=update_columns  # Use the dictionary of columns to update
                        ).whenNotMatchedInsertAll().execute()

                    except Exception as e:
                        print(delta_table_name)
                        df.printSchema()
                        print(e)
                    
                else:
                    # If the table does not exist, create it by writing the current DataFrame
                    # First, generate a UUID for all records in the new UUID column
                    df = df.withColumn(uuid_column_name, expr("uuid()"))

                    if ('studentkey' not in df.columns): # because we have already added organisationkey to that
                
                        # Load the dim_Organisation table to get the existing mappings
                        dim_org_df = spark.read.format("delta").load(f"{silver_path}/dim_Organisation").select("school_id", "organisationkey")

                        # Perform a left join to find existing organisation keys
                        df_joined = df.alias("source").join(
                            dim_org_df.alias("dim"),
                            col("source.school_id") == col("dim.school_id"),
                            "left"
                        )

                        # Select all columns from df and only the 'organisationkey' from the dim_Organisation
                        # Alias the dim_Organisation's organisationkey to avoid ambiguity
                        df_with_keys = df_joined.select("source.*", col("dim.organisationkey").alias("dim_organisationkey"))
                        df_with_keys.printSchema()

                        # Fill in the missing keys with UUIDs
                        # Ensure to use the aliased column name 'dim_organisationkey' to avoid ambiguity
                        df_complete = df_with_keys.withColumn(
                            "organisationkey",
                            when(col("dim_organisationkey").isNull(), expr("uuid()")).otherwise(col("dim_organisationkey"))
                        )

                        # Drop the 'dim_organisationkey' as it is no longer needed
                        df = df_complete.drop("dim_organisationkey")


                    # debug, show 20 records
                    df.show(n=20, truncate=False)

                    df.write.format("delta").mode("overwrite").save(silver_table_path)

StatementMeta(, 14833b89-9b01-4d0c-aad3-90400d1a9d55, 11, Finished, Available)

+----------+-------------+----------------+---------------+----------------+------+---------+-----------------+-------------------------------------------------------+-----------+---------------------------+---------------------+--------------------------+-------+------------------------------+-------------------+-----------------+----------------------+-----------------------+-------------+------------------------------------+--------------+-----------+------------------+------------+-------------+------------+-------+----------+------------------------------------+------------------------------------+
|student_id|school_id    |LEGAL_FIRST_NAME|LEGAL_LAST_NAME|Enrolment_Status|SEX   |Ethnicity|Religion         |First_Language                                         |Nationality|SEN_Status                 |Child_Protection_Plan|Gifted_And_Talented_Status|In_Care|English_As_Additional_Language|Free_School_Meals_6|Free_School_Meals|Pupil_Premium_Eligible|Pupil_Premium_Indicator|Service_C